<h1> Elo Merchant Category Recommendation :</h1>

<h2>Business Problem/Problem Statement :</h2>

> Elo Merchant Category Recommendation is a Kaggle competition which is provided by Elo. As a payment Brand, providing offer promotions and discounts with merchants is a good marketing strategy . Elo needs to keep their customers so loyalty of the customers towards the brand is crucial. For Example, a customer using the Elo card with diverse merchants for a long time, this signifies the user's loyalty is high. To keep the customer as a subscriber, Elo can run different promotional campaign’s targets towards customers with the customer’s favorite or frequently used merchants. These personalized reward programs are planned by the owners of the company to retain existing customers and attract new customers. So, the frequency of using their payment brand should increase. Basically, These programs make the customer’s choice more strongly towards the usage of Elo. The Problem is to find a metric which has to reflect the cardholder’s loyalty with Elo payment brand. Here we have the loyalty score which is a numerical score calculated 2 months after the historical and evaluation period. Elo uses it for their business decision about their promotional campaign.


<h2>Dataset Overview :</h2>

The datasets are largely anonymized, and the meaning of the features are not elaborated. External data is allowed.

The problem has 5 datasets.

> **train.csv :** It has 6 features, first_active_month, card-id, feature1, feature2, feature3 and target

> **test.csv :** The test set has the same features as the train set without targets

> **historical_transactions.csv :** Contains up to 3 months worth of historical transactions for each card_id

> **merchants.csv :** Contains the transactions at new merchants(merchant_ids that this particular card_id has not yet visited) over a period of two months.

> **new_merchant_transactions.csv :** Two months’ worth of data for each card_id containing ALL purchases that card_id made at merchant_ids that were not visited in the historical data

In all these datasets, no text data/feature is present. We only have categorical and numerical features. Additionally, by looking at historical_transactions.csv and new_merchant_transactions.csv, we can find that the historical transactions are the transactions occurred before the "reference date" and new merchant transactions - the ones that occurred after the reference date (according to the 'month_lag' field, which is generously described as "month lag to reference date").

<h2>Mapping the real-world problem to Machine Learning problem :</h2>

> In terms of Machine Learning, we need a metric to measure up the customer's loyalty.A certain loyalty score is assigned for each of the card_id present in train data.

>**Input Features —** Cardholder’s Purchase history, usage time etc.

>**Target Variable —** Loyalty Score

>The Loyalty Score is the target variable for which the Machine Learning Model should be built to predict. **What is loyalty?** According to the Data_Dictionary.xlsx, **loyalty is a numerical score calculated 2 months after historical and evaluation period.** The Loyalty score depends on many aspects of the customers. The purchase history, usage time, merchant’s diversity, etc.  Loyalty scores are real-numbers, It directly gives us the intuition that we have to go for a supervised machine learning regression model to solve this problem where features are as our input in train data and output is real number value which is our predicted loyalty score.

<h2>Real-world constraints :</h2>

> The constraint is that the data which has been provided is not real-customer data. The Provided data is Anonymous and simulated, I think this is due to privacy and legal constraints. Simulated data sometimes has an artificially induced bias which will affect the prediction model performance. We have to deal with this specifically.

<h2>Performance Metric :</h2>

> Root mean square error(RMSE) is used to evaluate our predictions with actual loyalty score. We want our predicted loyalty score close to the actual score. So we need to have a lower RMSE score. This gives us the knowledge that on the basis of input features how close our model makes the predictions as compared to actual predictions.


**My understanding of the problem :**

* Based on the data in historical_transactions.csv, Elo picked new mechants to recommend for each card holder.
* The date when Elo began providing recommentations is called the 'reference date'.
* The recommended mechant data is not provided (so we don't figure out the recommendation algorithm Elo uses).
* After the reference date, for each card Elo gathered transaction history for all new merchants that appeared on the card.
* By comparing each card's new merchant activity and the secret list of the merchants recommended by Elo, the loyalty score was calculated.
* **The goal is to evaluate Elo's recommendation algorithm by trying to predict in which cases it's going to work well (yielding a high loyalty score) and in which cases - not (yielding a low loyalty score).**

**Reduce the memory usage :**

In [ ]:
## Reference: https://www.kaggle.com/rinnqd/reduce-memory-usage

def reduce_memory_usage(df, verbose=True):
  '''
  This function reduces the memory sizes of dataframe by changing the datatypes of the columns.
  Parameters
  df - DataFrame whose size to be reduced
  verbose - Boolean, to mention the verbose required or not.
  '''
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df

**Import all the libraries :**

In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
sns.set_style("whitegrid")
import warnings 
warnings.simplefilter("ignore")
from statsmodels.stats.outliers_influence import variance_inflation_factor

**Loading Data :**


In [ ]:
train_data = pd.read_csv('../input/elo-merchant-category-recommendation/train.csv')
test_data = pd.read_csv('../input/elo-merchant-category-recommendation/test.csv')
historical_data = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv')
newmerchant_data = pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv')
merchants_data = pd.read_csv('../input/elo-merchant-category-recommendation/merchants.csv')

**Reduce memory usage of data :**

In [ ]:
train_data = reduce_memory_usage(train_data)
test_data = reduce_memory_usage(test_data)
historical_data = reduce_memory_usage(historical_data)
newmerchant_data = reduce_memory_usage(newmerchant_data)
merchants_data = reduce_memory_usage(merchants_data)

<h2>Exploring the train and test data files :</h2>

In [ ]:
# As our first file is excel file we have to read it with the excel command of pandas.
data_dictionary=pd.read_excel('../input/elo-merchant-category-recommendation/Data Dictionary.xlsx')
data_dictionary

**Observations :**

* This DataDictionary file have the description of all the features in Description column which were included in train.csv.

* From second row we have columns which have the description of all the columns in our data and third row tell us about the card_id and third one is about the first_active_month which tell us about the month and year of purchase of products.

* feature_1, feature_2, feature_3 has categorical value which is in row fourth,fifth,and sixth.

* last row tells us about the prediction on the basis of these features which is known as target column. or we can say loyalty score which is calculated after the two months.

In [ ]:
print('The number of rows in train_data is:',train_data.shape[0])
print('The number of rows in test_data is:',test_data.shape[0])
plt.bar([0,1],[train_data.shape[0],test_data.shape[0]])
plt.xticks([0,1],['train_rows','test_rows'])

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()
print("********************************************************************")
test_data.info()

**Obsaervations :**

* The main data train has 6 values. 'first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target'.
* first_active_month : This is active_month for card_id. 
* feature_1,2,3 : it is key important but hidden meaning.
* target : Loyalty numerical score calculated 2 months after historical and evaluation period
* We can infer that both the data have same columns and overall same structure. So, We will explore both data simultaneously.


**Missing values in train and test data :
(Check for nan values in the whole train and test data)**

In [ ]:
train_data.isna().any()

**Observation :** In train Data there is no nan values for any features in train data

In [ ]:
test_data.isna().any()

In [ ]:
test_data[test_data['first_active_month'].isna()]

**Observation :**
In the Test Data, there is one row with 'first_active_month' as nan value. Since it is test data we have to impute the value.

**Feature comparison in train and test data features :**

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5));
train_data['feature_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='feature_1');
train_data['feature_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='feature_2');
train_data['feature_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='feature_3');
plt.suptitle('Counts of categories for train features');

fig, ax = plt.subplots(1, 3, figsize = (15, 5));
test_data['feature_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='feature_1');
test_data['feature_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='feature_2');
test_data['feature_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='feature_3');
plt.suptitle('Counts of categories for test features');

**Observations :**

* We can see from above plots that test and train data are distributed similarly.
* feature_1, feature_2, feature_3, all are categorical variables
* feature_1 has 5 unique values
* feature_2 has 3 unique values
* feature_3 is a binary column

**Anonymised Features Analysis : feature_1, feature_2, feature_3**

**checking distributions with target :**

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(131)
sns.kdeplot(x ='target',data = train_data,hue = 'feature_1',palette='rainbow')
plt.title('distribution of target over different categories of Feature_1')
plt.subplot(132)
sns.kdeplot(x ='target',data = train_data,hue = 'feature_2',palette='Dark2_r')
plt.title('distribution of target over different categories of Feature_2')
plt.subplot(133)
sns.kdeplot(x ='target',data = train_data,hue = 'feature_3',palette='Dark2_r')
plt.title('distribution of target over different categories of Feature_3')
plt.show()

**Observations :** 

The above two plots show a key point : 

* while different categories of these features could have various counts, the distribution of target is almost the same. This could mean, that these features aren't really good at predicting target - we'll need other features and feature engineering. Also it is worth noticing that mean target values of each catogory of these features is near zero. This could mean that data was sampled from normal distribution.

**Note:** The same information can be gathered by using box-plot and violin-plot, I have tried all of them. Here, I use kdeplot as I found it more visually appealing. In further analysis I have used Box-plot more often.

**let's see Target column seperately :**

In [ ]:
train_data['target'].describe()

In [ ]:
#Plotting the pdf of target variable
sns.kdeplot(train_data['target'])
plt.title("PDF of Target")
plt.show()

**Observation:** The target value is almost normally distributed with bunch of outlier value near -30. This distribution indicates that the target value is normalized with pre-decided mean and standard deviation.

This outlier value of the target is a value which needs more look into the feature EDA to understand cause of it.

**Analyze the outliers :**



In [ ]:
loyality_score = train_data['target']
ax = loyality_score.plot.hist(bins=20, figsize=(6, 5))
_ = ax.set_title("target histogram")
plt.show()

fig, axs = plt.subplots(1,2, figsize=(12, 5))
_ = loyality_score[loyality_score > 10].plot.hist(ax=axs[0])
_ = axs[0].set_title("target histogram for values greater than 10")
_ = loyality_score[loyality_score < -10].plot.hist(ax=axs[1])
_ = axs[1].set_title("target histogram for values less than -10")
plt.show()


**Observations :**

* Values range from -33.2 to 17.9

* -33 seems like an outlier as can be seen in the 3rd plot

* other values less than -10 also seem like outliers due to very less in number

* All values above 10 are also looking like outliers

In [ ]:
target_sign = loyality_score.apply(lambda x: 0 if x <= 0 else 1)
target_sign.value_counts()

**Observation :** Negative and positive target values are almost in the same proportion

In [ ]:
outliers_in_target= train_data.loc[(train_data['target']< -10) | (train_data['target']>10)]
print(' The number of outliers in the data is:',outliers_in_target.shape[0])
non_outliers_in_target= train_data.loc[(train_data['target'] >=-10) & (train_data['target']<=10)]
print(' The number of non-outliers in the data is:',non_outliers_in_target.shape[0])

Outliers comparison with the feature of target :

In [ ]:
plt.figure(figsize=[16,9])
plt.suptitle('Outlier vs. non-outlier feature distributions', fontsize=20, y=1.1)

for num, col in enumerate(['feature_1', 'feature_2', 'feature_3', 'target']):
    if col is not 'target':
        plt.subplot(2, 3, num+1)
        non_outlier = non_outliers_in_target[col].value_counts() / non_outliers_in_target.shape[0]
        plt.bar(non_outlier.index, non_outlier, label=('non-outliers'), align='edge', width=-0.3, edgecolor=[0.2]*3,color=['teal'])
        outlier = outliers_in_target[col].value_counts() / outliers_in_target.shape[0]
        plt.bar(outlier.index, outlier, label=('outliers'), align='edge', width=0.3, edgecolor=[0.2]*3,color=['brown'])
        plt.title(col)
        plt.legend()

plt.tight_layout()
plt.show()

**Observations :**

* We can see There are only slight differences between outliers and non-outliers, but they don't seem to be that big and they certainly can't explain the difference between the target values, at least based on the features in the train dataset. It means the card_id's having outliers as loyality score having pretty much similar properties to the regular ones.

* Outliers could be one of the main purposes of this competition. May be those represent fraud or credit default etc. i.e. they are important. The target variable is normally distributed, and outliers seem to be purposely introduced in the loyalty formula. 

* As noted in multiple threads over kaggle, more than half of the RMSE is due to the outliers with loyalty scores of ~ -33. They strongly mention, If we try to replace these outliers with the median, retrain the model and submit, we will
find our leaderboard score WORSE than if we keep the outliers at their original values. Impute any values will significantly affect the RMSE score for test set. So, imputations have been excluded. This tells us that outliers are included in the test set. Furthermore, given the magnitude of the impact of outliers on the RMSE score, Our focus should be on predicting those outliers as accurately as possible.

* For mitigating the impact of outliers, We can make the outliers as a binary feature whether card's target value is outliers or not. So that while training our model can learn that given entry has target score as outlier or not and use this information while predicting loyality score.

**Analysis of feature First_active_month :**

Distribution of first_active_month across years :

In [ ]:
year_train = train_data['first_active_month'].value_counts().sort_index()
year_test = test_data['first_active_month'].value_counts().sort_index()
ax = year_train.plot(figsize=(10, 5))
ax = year_test.plot(figsize=(10, 5))
_ = ax.set_xticklabels(range(2010, 2020))
_ = ax.set_title("Distribution across years")
_ = ax.legend(['train', 'test'])

**Observation :** Years range from 2011 to 2018. But, Most of the data lies in the years ranging from 2016 to 2018 and trends of counts for train and test data are similar.

Distribution of first_active_month across months :

In [ ]:
train_data["month"] = train_data['first_active_month'].str.split("-").str[1]
train_data.head()

In [ ]:
temp = train_data['month'].value_counts().sort_index()
ax = temp.plot()
_ = ax.set_xticklabels(range(-1, 15, 2))
_ = ax.set_title("Distribution across months")

**Observations :** Last 6 months (July to December) has relatively more data than first 6 months (January to June).

**First_active_month Vs Target variable :**

In [ ]:
train_data['first_active_month'] = pd.to_datetime(train_data['first_active_month'],
                                             format='%Y-%m')

In [ ]:
sns.lineplot(x = train_data['first_active_month'], y= train_data['target'])
plt.title("Distribution of target over first_active_month")
plt.show()

**Observations :**

* The above plot reveals that the target variable (loyalty score) behaves like a damping frequency plot. And it is mentioned in the Buisness problem that the target score is calcuated with the recent year transactions.

* Older Card's: The cards which have first active month from 2012 to 2015.

* new card's: The cards which have first active month from 2015 to 2018.

* The Older card's have large number of transactions which affects the target towards the negative value. and the new card's have transactions which affects the target towards positive value.

So, I think the type of transactions by the newer card's is different from the older card's which helps in increase the loyalty Score.



**Correlation between variables : Variance Inflation Factor**

In [ ]:
#Finding Correlation between variables of train_data features
selected_columns = ['feature_1','feature_2','feature_3']
data_frame = train_data[selected_columns]

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

In [ ]:
#Finding Correlation between variables of test_data features
selected_columns = ['feature_1','feature_2','feature_3']
data_frame = test_data[selected_columns]

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

**Observations :** 

* The VIF values for all the three features are well under 10. So, there is no problem of multicollinearity in the train data and test data.
* Also VIF values are very near to 0, that interprates fetures are not at all correlated.

<h2>Exploring the historical_transactions and new_merchant_transactions data files :</h2>

In [ ]:
data_dictionary = pd.read_excel('../input/elo-merchant-category-recommendation/Data_Dictionary.xlsx', sheet_name='history')
data_dictionary

In [ ]:

data_dictionary = pd.read_excel('../input/elo-merchant-category-recommendation/Data_Dictionary.xlsx', sheet_name='new_merchant_period')
data_dictionary

**Observation :** After going through Data Dictionary.xlsx, We can infer that both the data have same columns and overall same structure. So, We will Explore both data side by side.

In [ ]:
print(f'{historical_data.shape[0]} rows in data\n')
historical_data.head()

In [ ]:
print(f'{newmerchant_data.shape[0]} rows in data\n')
newmerchant_data.head()

In [ ]:
historical_data.info()

In [ ]:
newmerchant_data.info()

**Observations :**

We can see that there are:

* 6 features type ID: card_id, merchant_category_id, subsector_id, merchant_id, city_id, state_id

* 2 features type integer/counter: month_lag, installments

* 1 feature type numerical: purchase_amount

* 1 feature type date: purchase_date

* 4 features type categorical: authorized_flag, category_3, category_1, category_2

In [ ]:
historical_data.isna().any()

In [ ]:
newmerchant_data.isna().any()

**Observations :** Both historical_transaction and new_merchant_transaction have Nan values in same columns which are : merchand_id, category_2, category_3.

**Analysis of Category Features : category_1,category_2 and category_3**

In [ ]:
print('Value counts for category features of Historical Transactions :\n')
print(historical_data['category_1'].value_counts())
print('*****************************')
print(historical_data['category_2'].value_counts())
print('*****************************')
print(historical_data['category_3'].value_counts())

print('\nValue counts for category features of New merchant Transactions :\n')
print(newmerchant_data['category_1'].value_counts())
print('*****************************')
print(newmerchant_data['category_2'].value_counts())
print('*****************************')
print(newmerchant_data['category_3'].value_counts())



In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5));
historical_data['category_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='category_1');
historical_data['category_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='category_2');
historical_data['category_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='category_3');
plt.suptitle('Counts for category features of Historical Transactions New merchant Transactions');


fig, ax = plt.subplots(1, 3, figsize = (15, 5));
newmerchant_data['category_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='category_1');
newmerchant_data['category_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='category_2');
newmerchant_data['category_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='category_3');
plt.suptitle('Counts for category features of New merchant Transactions');

**Observation :** The distribution of these three category features are almost 
identical in historical and new transactions.This shows these Category feature represent the inately charcterstics of the transactions which is constant over the period.So, these features can be an importance feature in the decision function on final model.

**Distrbution of target over categorical features :**

**Note :** The train.csv file only has the target value, which is the feature we are gonna predict with models build in the future But, transactions data don't have the target values in it for each card_id's. By merging the "target" feature with the transactions data will help in Data analysis to fully understand different fetures in transactional dataFrame.

In [ ]:
# merging target value of card_id for each transction in historical_transactions Data
historical_data = pd.merge(historical_data, train_data[['card_id','target']], how = 'outer', on = 'card_id')

# merging target value of card_id for each transction in new_merchants_transactions Data
newmerchant_data = pd.merge(newmerchant_data, train_data[['card_id','target']], how = 'outer', on = 'card_id')

In [ ]:
historical_data.head()

In [ ]:
newmerchant_data.head()

In [ ]:
plt.figure(figsize = (20,10))
plt.subplot(231)
sns.kdeplot(x ='target',data = historical_data,hue = 'category_1',palette='Dark2_r')
plt.title("Distribution of target over Category_1 in historical data")
plt.subplot(232)
sns.kdeplot(x ='target',data = historical_data,hue = 'category_2',palette='Dark2_r')
plt.title("Distribution of target over Category_2 in historical data")
plt.subplot(233)
sns.kdeplot(x ='target',data = historical_data,hue = 'category_3',palette='rainbow')
plt.title("Distribution of target over Category_3 in historical data")
plt.subplot(234)
sns.kdeplot(x ='target',data = newmerchant_data,hue = 'category_1',palette='Dark2_r')
plt.title("Distribution of target over Category_1 in new_merchent data")
plt.subplot(235)
sns.kdeplot(x ='target',data = newmerchant_data,hue = 'category_2',palette='Dark2_r')
plt.title("Distribution of target over Category_2 in new_merchent data")
plt.subplot(236)
sns.kdeplot(x ='target',data = newmerchant_data,hue = 'category_3',palette='rainbow')
plt.title("Distribution of target over Category_3 in new_merchent data")
plt.tight_layout()
plt.show()

**Observations :**

* These three category features doesn't explicity help to differentiate the target Score(Loyalty Score). Every category have outliers in each of the sub_categories. And Almost all the category have Same IQR range.

* These anonymous features doesn't reveal any important info for further feature engineering of these categories.

**Note:** The same information can be gathered by using box-plot and violin-plot, I have tried all of them. Here, I use kdeplot as I found it more visually appealing. In further analysis I have used Box-plot more often.


**Authorized Flag Feature Analysis :**

In [ ]:
print('Value counts for Authorized Flag of Historical Transactions :')
print(historical_data['authorized_flag'].value_counts())
print('*************************************************************')
print('Value counts for Authorized Flag of New Merchant Transactions :')
print(newmerchant_data['authorized_flag'].value_counts())

#barplot for the authorized_flag feature
fig, ax = plt.subplots(1, 2, figsize = (12, 5));
historical_data['authorized_flag'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='\nauthorized_flag(historical_transactions)');
newmerchant_data['authorized_flag'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='\n   authorized_flag(new_merchant_transactions)');

**Observations :**

* The new transactions have no "N" category in authorized_flag. This historical transactions have both "Y" and "N".

* The authorized_flag 'Y' if approved, 'N' if denied - whether the transaction is approved or Denied.

* If we calculate percentage of authorized transaction in historical transaction. At average 91.3545% transactions are authorized.

* This feature is an important feature for predicting the Loyalty score. because, if the card's transactions are approved most of time, there is a great chance the cards can have high Loyalty Score

Distributions of target over authorized flag :

In [ ]:
plt.figure(figsize = (14,5))
plt.subplot(121)
sns.boxplot(y = 'target',x= 'authorized_flag', data = historical_data)
plt.title("Distrbutions of target over authorized flag(historical_transactions)")
plt.subplot(122)
sns.boxplot(y = 'target',x= 'authorized_flag', data = newmerchant_data)
plt.title("Distrbutions of target over authorized flag(new_merchant_transactions)")
plt.tight_layout()
plt.show()

**Observations :** 

* The authorized Flag also doesn't give a suspectble change in the IQR range between authorized and un_authorized transactions.

* Even for the un_authorized transactions card users have Same IQR. Because of the many transactions by an user, these un_authorized doesn't have much effect.

* But this categorical features also should be included using response coding.

**Analysis of installments feature :**

In [ ]:
print('Quantile values for installments in Historical Transaction :')
print('25th Percentile :',historical_data['installments'].quantile(0.25))
print('50th Percentile :',historical_data['installments'].quantile(0.50))
print('75th Percentile :',historical_data['installments'].quantile(0.75))
print('100th Percentile :',historical_data['installments'].quantile(1))
print('\n******************************************************************\n')
print('Quantile values for installments in New Merchant Transaction :')
print('25th Percentile :',newmerchant_data['installments'].quantile(0.25))
print('50th Percentile :',newmerchant_data['installments'].quantile(0.50))
print('75th Percentile :',newmerchant_data['installments'].quantile(0.75))
print('100th Percentile :',newmerchant_data['installments'].quantile(1))

Distribution of target over installment feature :

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
sns.boxplot(y='target',x= 'installments', data = historical_data)
plt.title("Distrbutions of target over installments(historical_transactions)")
plt.subplot(122)
sns.boxplot(y='target',x = 'installments', data = newmerchant_data)
plt.title("Distrbutions of target over installments(new_merchant_transactions)")
plt.tight_layout()
plt.show()

**Observations :** The installments also have outliers, these outliers should be taken care in data preprocessing. In historical_transactions and new_merchants_transactions the 75% of installments are below 1. So, most of the payments through the cards are instant payments or short term installments.

**Analysis of purchase_amount feature :**

In [ ]:
print('Quantile values for purchase amount in Historical Transaction :')
print('25th Percentile :',historical_data['purchase_amount'].quantile(0.25))
print('50th Percentile :',historical_data['purchase_amount'].quantile(0.50))
print('75th Percentile :',historical_data['purchase_amount'].quantile(0.75))
print('100th Percentile :',historical_data['purchase_amount'].quantile(1))
print('\n******************************************************************\n')
print('Quantile values for purchase amount in New Merchant Transaction :')
print('25th Percentile :',newmerchant_data['purchase_amount'].quantile(0.25))
print('50th Percentile :',newmerchant_data['purchase_amount'].quantile(0.50))
print('75th Percentile :',newmerchant_data['purchase_amount'].quantile(0.75))
print('100th Percentile :',newmerchant_data['purchase_amount'].quantile(1))

**Observation :** The IQR range value is very small. And there is one outlier which have 6010603.9717525. These outlier can skew the final model performance. purchase_amount is normalized. Let's have a look at it nevertheless.

In [ ]:
plt.figure(figsize = (13,5))
plt.subplot(121)
plt.title('Purchase amount(Historical Transaction)');
historical_data['purchase_amount'].plot(kind='hist');
plt.subplot(122)
plt.title('Purchase amount(NewMerchant Transaction)');
newmerchant_data['purchase_amount'].plot(kind='hist');

In [ ]:
print('For purchase_amount in Historical transactions :')
for i in [-1, 0]:
    n = historical_data.loc[historical_data['purchase_amount'] < i].shape[0]
    print(f"There are {n} transactions with purchase_amount less than {i}.")
for i in [0, 10, 100]:
    n = historical_data.loc[historical_data['purchase_amount'] > i].shape[0]
    print(f"There are {n} transactions with purchase_amount more than {i}.")
    
print('\n******************************************************************\n')
print('For purchase_amount in New Merchant transactions :')
for i in [-1, 0]:
    n = newmerchant_data.loc[newmerchant_data['purchase_amount'] < i].shape[0]
    print(f"There are {n} transactions with purchase_amount less than {i}.")
for i in [0, 10, 100]:
    n = newmerchant_data.loc[newmerchant_data['purchase_amount'] > i].shape[0]
    print(f"There are {n} transactions with purchase_amount more than {i}.")

**Observation :** As we can see the major chunk of transactions has purchase_amount less than 0. let us see Purchase amount distribution for negative values.

In [ ]:
plt.figure(figsize = (14,5))
plt.subplot(121)
plt.title(' Negative purchase_amount distribution(Historical)');
historical_data.loc[historical_data['purchase_amount'] < 0, 'purchase_amount'].plot(kind='hist');
plt.subplot(122)
plt.title('Negative purchase_amount distribution(New Merchant)');
newmerchant_data.loc[newmerchant_data['purchase_amount'] < 0, 'purchase_amount'].plot(kind='hist');

**Observation :** It seems that almost all transactions have purchase amount in range (-1, 0). Quite a strong normalization and high outliers, which will need to be processed.

Now, let's see purchase_amount feature over target variable :



In [ ]:
#There is one outlier which have value 6010603.9717525. So, I remove that for EDA part.
historical_data = historical_data[historical_data['purchase_amount']  != 6010603.9717525]

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(121)
sns.scatterplot(data=historical_data, x="purchase_amount", y="target")
plt.title("purchase_amount(historical_transaction) over target")
plt.subplot(122)
sns.scatterplot(data=newmerchant_data, x="purchase_amount", y="target")
plt.title("purchase_amount(newmerchant_transaction) over target")
plt.tight_layout()
plt.show()

**Observations :**

* One key observation here is, Most of the outliers in target having value around -30 are having very less purchase amount.
* With the increase in purchase amount customer become more loyal, as target score increases.

**Analysis of feature Month_lag :**

In [ ]:
plt.figure(figsize = (13,5))
plt.subplot(121)
plt.title('Month lag(Historical Transaction)');
historical_data['month_lag'].plot(kind='hist');
plt.subplot(122)
plt.title('Month lag(NewMerchant Transaction)');
newmerchant_data['month_lag'].plot(kind='hist');

Distribution of target over month_lag feature :

In [ ]:
plt.figure(figsize = (14,5))
plt.subplot(121)
sns.boxplot(y= 'target',x= 'month_lag', data = historical_data)
plt.title("Distrbutions of target over month_lag(historical_transactions)")

plt.subplot(122)
sns.boxplot(y= 'target',x= 'month_lag', data = newmerchant_data)
plt.title("Distrbutions of target over month_lag(historical_transactions)")
plt.tight_layout()
plt.show()

**Observations :** 

* The Month_lag gives important info to predict the loyalty score. For a Purchase in installments, how many months the card lags from the actual end date of installment is the month_lag feature.

* The historical_transactions have month_lags from 0 to 13. which means the cards with transactions in histortical_transactions data have lag of installments from 0 to 13. But, the new_merchant_transactions have month_lag 1 and 2 only.

* This again proves the difference in the transactions type between the historical and new merchants.

**Analysis of feture purchase_date :**

At first, we convert purchase_date to datetime format :

In [ ]:
historical_data['purchase_date'] = pd.to_datetime(historical_data['purchase_date'],
                                             format='%Y-%m-%d %H:%M:%S')
newmerchant_data['purchase_date'] = pd.to_datetime(newmerchant_data['purchase_date'],
                                             format='%Y-%m-%d %H:%M:%S')

Number of transactions vs Year :

In [ ]:
#barplot for the Number of transactions vs Year
fig, ax = plt.subplots(1, 2, figsize = (14, 5));
historical_data['purchase_date'].dt.year.value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='\n   Transactions Vs Year(histortical_transactions)');
newmerchant_data['purchase_date'].dt.year.value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='\n   Transactions Vs Year(new_merchant transactions)');


print('Year-Wise Percentage distribution of purchase_date(Historical-Transaction) :')
print(historical_data['purchase_date'].dt.year.value_counts(normalize = True)*100)
print('\nYear-Wise Percentage distribution of purchase_date(NewMerchant-Transaction) :')
print(newmerchant_data['purchase_date'].dt.year.value_counts(normalize = True)*100)

**Observations :**

* In historical_transactions, The transactions with respect to year 2017 is way more (~82%) than transactions in 2018 (18%). 

* But, In new_merchant_transactions, transactions with respect to 2018 is way more (~85%) than transactions in 2018 (15%).

* Then we can say, new_merchant_transactions are the recent year transactions. This is the reason for the disparity in the purchase amount and installment features.

Number of transactions vs Week

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 5));
historical_data['purchase_date'].dt.dayofweek.value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='Transactions Vs dayofweek(histortical_transactions)');
newmerchant_data['purchase_date'].dt.dayofweek.value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='Transactions Vs dayofweek(new_merchant_transactions)');

Distribution of target over dayofweek :

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
sns.boxplot(y = historical_data['target'], x = historical_data['purchase_date'].dt.dayofweek)
plt.xticks(range(0,7),labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.xlabel('Days of week')
plt.title("Distribution of target over dayofweek(histortical_transactions)")

plt.subplot(122)
sns.boxplot(y = historical_data['target'], x = newmerchant_data['purchase_date'].dt.dayofweek)
plt.xticks(range(0,7),labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.xlabel('Days of week')
plt.title("Distribution of target over dayofweek(new_merchant_transactions)")
plt.tight_layout()
plt.show()

Number of transactions vs hour

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 5));
historical_data['purchase_date'].dt.hour.value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='Transactions Vs hour(histortical_transactions)');
newmerchant_data['purchase_date'].dt.hour.value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='Transactions Vs hour(new_merchant_transactions)');

Distribution of target over hour :

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
sns.boxplot(y = historical_data['target'], x = historical_data['purchase_date'].dt.hour)
plt.xlabel('Hour')
plt.xticks(range(0,24))
plt.title("Distribution of target over hour(histortical_transactions)")

plt.subplot(122)
sns.boxplot(y = historical_data['target'], x = newmerchant_data['purchase_date'].dt.hour)
plt.xlabel('Hour')
plt.xticks(range(0,24))
plt.title("Distribution of target over hour(new_merchant_transactions)")
plt.tight_layout()
plt.show()


**Observations :**

* From the distribution of both weekly and hourly transactions count, these transactions have not much difference in their distributions.

* Since, the data given in the problem is a generated data and not a real time data. The distribution of the transactions over the purchase date is similar.

* But, the type of transactions differs from historical and new_merchants in terms of purchase_amount,month_lag and installments.

* By checking the number of merchants are in both historical and new_merchants transactions, we can get exclusive informations of the merchants.

**Let's create a feature called Number of transactions for each card_id and see - How it impacts target variable ?**

Number of Transactions feature is not explicitly given in any of the file but we can derive it with some hacks :

In [ ]:
#For historical transaction
g = historical_data[['card_id']].groupby('card_id')
df_transaction_counts = g.size().reset_index(name='num_transactions')
historical_data = pd.merge(historical_data ,df_transaction_counts, on="card_id",how='left')
historical_data.head()

In [ ]:
historical_data['num_transactions'].describe()

In [ ]:
#For New Merchant transaction
g = newmerchant_data[['card_id']].groupby('card_id')
df_transaction_counts = g.size().reset_index(name='num_transactions')
newmerchant_data = pd.merge(newmerchant_data ,df_transaction_counts, on="card_id",how='left')
newmerchant_data.head()

In [ ]:
newmerchant_data['num_transactions'].describe()

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(121)
sns.scatterplot(data=historical_data, x="num_transactions", y="target")
plt.title("Number of transactions(historical_transaction) VS target")
plt.subplot(122)
sns.scatterplot(data=newmerchant_data, x="num_transactions", y="target")
plt.title("Number of Transactions(newmerchant_transaction) VS target")
plt.tight_layout()
plt.show()

**Observations :**

* One key observation here is, Most of the outliers in target having value around -30 are having very less no of transactions.
* With increase in no of transactions customer become more loyal, as target score increases.

**Correlation between variables : Variance Inflation Factor**

In [ ]:
selected_columns = ['category_2','month_lag','purchase_amount','state_id','subsector_id', 'installments']
data_frame = newmerchant_data[selected_columns]

data_frame = data_frame.dropna()

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

All values are under 10, let's add some more features and again we'll calculate the VIF :




In [ ]:
Dict = {'A':1,'B':2,'C':3}
Dict1 = {'Y':1,'N':0}

selected_columns = ['authorized_flag','category_3','category_2','month_lag','purchase_amount','state_id','subsector_id', 'installments']
data_frame = newmerchant_data[selected_columns]
data_frame['category_3'] = data_frame['category_3'].map(Dict)
data_frame['authorized_flag'] = data_frame['authorized_flag'].map(Dict1)

data_frame = data_frame.dropna()

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

**Observations :**

* The value for the authorized flag is somewhat higher, it is around 32 which indicates possible correlation. So this variable needs further investigation.

* Other than the authorized flag the remaining variables doesn't look correlated. They are well under 2.

<h2>Exploring the Merchant Data :</h2>



In [ ]:
data_dictionary = pd.read_excel('../input/elo-merchant-category-recommendation/Data_Dictionary.xlsx', sheet_name='merchant')
data_dictionary

In [ ]:
merchants_data.head()

In [ ]:
merchants_data.info()

In [ ]:
merchants_data.isna().any()

**Observations :** Merchant data has missing values in columns : avg_sales_lag3, avg_sales_lag6 and avg_sales_lag12 

**Analysis of Numerical features : numerical_1 and numerical_2**

In [ ]:
print('Quantile values for numeric_1 in Transaction data:')
print('25th Percentile :',merchants_data['numerical_1'].quantile(0.25))
print('50th Percentile :',merchants_data['numerical_1'].quantile(0.50))
print('75th Percentile :',merchants_data['numerical_1'].quantile(0.75))
print('100th Percentile :',merchants_data['numerical_1'].quantile(1))
print('\n******************************************************************\n')
print('Quantile values for numeric_2 in Transaction data:')
print('25th Percentile :',merchants_data['numerical_2'].quantile(0.25))
print('50th Percentile :',merchants_data['numerical_2'].quantile(0.50))
print('75th Percentile :',merchants_data['numerical_2'].quantile(0.75))
print('100th Percentile :',merchants_data['numerical_2'].quantile(1))

**Observation :** I think the Distribution of numerical_1 and numerical_2 featurs are almost identical, because three qunatiles have identical values.

In [ ]:
plt.figure(figsize=(12,5) )
plt.subplot(121)
sns.kdeplot(np.log10(merchants_data['numerical_1']),shade=True)
plt.title("PDF of numerical_1 in LogScale")
plt.xlabel('log(numerical_1)')
plt.subplot(122)
sns.kdeplot(np.log10(merchants_data['numerical_2']),shade=True)
plt.title("PDF of numerical_2 in LogScale")
plt.xlabel('log(numerical_2)')
plt.tight_layout()
plt.show()

**Observation :** After plotting PDF, it is very clear that both the features have same distribution, may be they are duplicates of each other.

**Note :** The values for numeric_1 and numeric_2 are mostly -ve and very near to zero. So, I preferred LogScale for analysis.

**Analysis of the three anonymized category features : category_1,category_2 and category_4**

In [ ]:
print('Value counts for category features of Merchants data :\n')
print(merchants_data['category_1'].value_counts())
print('******************************')
print(merchants_data['category_2'].value_counts())
print('******************************')
print(merchants_data['category_4'].value_counts())

fig, ax = plt.subplots(1, 3, figsize = (15, 5));
merchants_data['category_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='category_1');
merchants_data['category_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='category_2');
merchants_data['category_4'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='category_3');
plt.suptitle('Counts for category features of Merchants_data');
             


**Observation :** These are anonymous categories, which can represent some properties of the merchants, which is still unclear after merging with the transactions data it can reveal more info.

**Analysis of feture most_recent_sales_range and most_recent_purchases_range :**

In [ ]:
print(merchants_data['most_recent_sales_range'].value_counts())
print('*******************************************')
print(merchants_data['most_recent_purchases_range'].value_counts())

fig, ax = plt.subplots(1, 2, figsize = (10, 5));
merchants_data['most_recent_sales_range'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='most_recent_sales_range');
merchants_data['most_recent_purchases_range'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='most_recent_purchases_range');

**Observations :**

* Both the features have very similar distributions.

* The sales range in last active month is a categorical feature with "A","B","C","D","E". after observing the trend from graph we can say Range of revenue (monetary units) is in order E > D > C > B > A.

* The Bar Plot shows there are many merchants with revenue range of "E" than other ranges.

* And also, Bar Plot shows there are many merchants with purchase quantity range of "E" than other ranges.

* The sales range and purchase range can be used in aggregated to know the card_id's most visited merchants in the final features for training.

**Analysis of Sales Average features : avg_sales_lag3,avg_sales_lag3,avg_sales_lag3,avg_purchases_lag6,avg_purchases_lag6 and avg_purchases_lag6**


In [ ]:
print('Quantile values for avg_sales_lag3 in Transaction data:')
print('25th Percentile :',merchants_data['avg_sales_lag3'].quantile(0.25))
print('50th Percentile :',merchants_data['avg_sales_lag3'].quantile(0.50))
print('75th Percentile :',merchants_data['avg_sales_lag3'].quantile(0.75))
print('100th Percentile :',merchants_data['avg_sales_lag3'].quantile(1))
print('\n******************************************************************\n')
print('Quantile values for avg_sales_lag6 in Transaction data:')
print('25th Percentile :',merchants_data['avg_sales_lag6'].quantile(0.25))
print('50th Percentile :',merchants_data['avg_sales_lag6'].quantile(0.50))
print('75th Percentile :',merchants_data['avg_sales_lag6'].quantile(0.75))
print('100th Percentile :',merchants_data['avg_sales_lag6'].quantile(1))
print('Quantile values for numeric_1 in Transaction data6:')
print('\n******************************************************************\n')
print('Quantile values for avg_sales_lag12 in Transaction data:')
print('25th Percentile :',merchants_data['avg_sales_lag12'].quantile(0.25))
print('50th Percentile :',merchants_data['avg_sales_lag12'].quantile(0.50))
print('75th Percentile :',merchants_data['avg_sales_lag12'].quantile(0.75))
print('100th Percentile :',merchants_data['avg_sales_lag12'].quantile(1))

In [ ]:
print('Statistical insights for avg_purchases_lag3 in Transaction data:')
print(merchants_data['avg_purchases_lag3'].describe())
print('\n******************************************************************\n')
print('Statistical insights for avg_purchases_lag6 in Transaction data:')
print(merchants_data['avg_purchases_lag6'].describe())
print('\n******************************************************************\n')
print('Statistical insights for avg_purchases_lag12 in Transaction data:')
print(merchants_data['avg_purchases_lag12'].describe())

**Observation :** There are outliers with the value inf in each of these columns, we have to deal with it. For EDA part, I am removing the corrosponding rows with the inf values in the columns avg_purchases_lag3,avg_purchases_lag6,avg_purchases_lag12. We will se what else we can do with these outliers in preprocessing part.

In [ ]:
merchants_data = merchants_data[merchants_data['avg_purchases_lag3']  != np.inf]
merchants_data = merchants_data[merchants_data['avg_purchases_lag6']  != np.inf]
merchants_data = merchants_data[merchants_data['avg_purchases_lag12']  != np.inf]

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(231)
sns.kdeplot(np.log10(merchants_data['avg_sales_lag3']),shade=True)
plt.title("PDF of avg_sales_lag3 in LogScale")
plt.xlabel('log(avg_sales_lag3)')
plt.subplot(232)
sns.kdeplot(np.log10(merchants_data['avg_sales_lag6']),shade=True)
plt.title("PDF of avg_sales_lag6 in LogScale")
plt.xlabel('log(avg_sales_lag6)')
plt.subplot(233)
sns.kdeplot(np.log10(merchants_data['avg_sales_lag12']),shade=True)
plt.title("PDF of avg_sales_lag12 in LogScale")
plt.xlabel('log(avg_sales_lag12)')
plt.subplot(234)
sns.kdeplot(np.log10(merchants_data['avg_purchases_lag3']),shade=True)
plt.title("PDF of avg_purchases_lag3 in LogScale")
plt.xlabel('log(avg_purchases_lag3)')
plt.subplot(235)
sns.kdeplot(np.log10(merchants_data['avg_purchases_lag6']),shade=True)
plt.title("PDF of avg_purchases_lag6 in LogScale")
plt.xlabel('log(avg_purchases_lag6)')
plt.subplot(236)
sns.kdeplot(np.log10(merchants_data['avg_purchases_lag12']),shade=True)
plt.title("PDF of avg_purchases_lag12 in LogScale")
plt.xlabel('log(avg_purchases_lag12)')
plt.tight_layout()
plt.show()

**Observations :** 

* The average purchases and sales across 3,6 and 12 months are distributed near 1.

* And, there are outliers in all the average sales and purchases. These features gives info about the merchants but not about the card_id's. The information about the merchants have to cumulated for each card_id's.

**Note :** The values for All the sales features listed above are mostly surrounded very near to 1. So, I preferred LogScale for analysis.

**Quantity of active months : Analysis of features(active_months_lag3,active_months_lag6 and active_months_lag3) :**

In [ ]:
print(merchants_data['active_months_lag3'].value_counts())
print('**********************************')
print(merchants_data['active_months_lag6'].value_counts())
print('**********************************')
print(merchants_data['active_months_lag12'].value_counts())

fig, ax = plt.subplots(1, 3, figsize = (15, 5));
merchants_data['active_months_lag3'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='active_months_lag3');
merchants_data['active_months_lag6'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='active_months_lag6');
merchants_data['active_months_lag12'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='active_months_lag12');
plt.suptitle('Counts of Active month lags');

**Observations :** The active months features are greatly skewed and doesn't provide any vital information about the cards.

**Correlation between variables : Variance Inflation Factor**

In [ ]:
selected_columns = ['numerical_1', 'numerical_2','category_2','avg_sales_lag3','avg_sales_lag6','avg_sales_lag12','avg_purchases_lag3','avg_purchases_lag6','avg_purchases_lag12','active_months_lag3','active_months_lag6','active_months_lag12']
data_frame = merchants_data[selected_columns]
data_frame = data_frame.dropna()

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

**Observation :** Looks like there are variables which are heavily correlated like active_months_lag6, avg_purchase_lag6 and avg_sales_lag_6 and avg_purchase_lag12 and as we seen before the numerical_1 and numerical_2 have similar values and distributions and they are correlated.



**TOTAL OBSERVATIONS :**

1) Target variable i.e. Loyalty scores are real-numbers, It directly gives us the intuition that we have to go for a supervised machine learning regression model to solve this problem.

2) The data files are train, test, new_merchant, merchant and historical transactions. but datasets are largely anonymized, and the meaning of the features are not elaborated.

3) The dimensionality of train and test data is very less. That clearly shows that the information provided is not sufficient for training. As only three features have been given in the train file which seems to be not sufficient to make good predictions. More features must be added to this with the help of domain knowledge and the business problem given.

4) Distribution of both the train and test are almost identical. So there is no time based splitting in the make over of the data. And, it assures for prediction of the test data.

5) The target variable is normally distributed but, there are outliers which seems to be accumulated around -30.

6) Data is not complete as nan values are present in the merchants, historical and new merchants transactions, so these missing values must be imputed for better predciton.

7) One-hot encoding/response coding of categorical features should be done for better prediction. The categorical features present across dataset are large in number than numerical features. 

8) Merchants data have high number of correlated features in it as compared to other data files. This is suggested by the calcuation of the VIF Scores 

9) The time features can reveal the inherent property of the transactions and the transactions are time dependent, the engineered features from the features like puchase_date will be useful in prediction.

10) In the historical transactions data theres is this feature called authorized_flag count which indicates whether the transaction is authorized or not. There is very less number of transactions which is not authorized. Considering this flag features as a seperater in the feature engineering can results can give better predction.

At the End of the Exploration of the transactions, merchants and train data, the given features of transactions are not big factor for the caculation of the target Score.

There exist an aggregrated or engineered feature or features which can be helpful in predicting the target Score.

With the different feature engineering techniques and market research techniques we have to produce the new feaures which may or may not be very useful in the predcition model.

By implementing the major feature engineering ideas we have to produce features and build model upon it.